# IMPORT DES DONNÉES

## Déclaration de fonction utilitaire pour le téléchargement et prétraitement des données

In [69]:
import requests
import shutil
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0"

PROJECT_DATA_DIR = "../../data/projet"
SUB_DATA_DIR = "/import"

def download_images(row):
    url, save_path = row["origin_path"], row["local_path"]
    headers = {
        "User-Agent": "Mozilla/5.0",
    }

    if not save_path:
        save_path = os.path.basename(url)

    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)

    if request.status_code != 200:
        print(f"Erreur lors du téléchargement des images {request.status_code}")
    else:
        with open(save_path, "wb") as image:
            request.raw.decode_content = True

            shutil.copyfileobj(request.raw, image)
            print("> Image sauvegardé :", save_path)
    print("Téléchargement de l'image :", row["local_path"])


def format_data(raw_data):
    def format_import_data_path(image_id):
        return f"{PROJECT_DATA_DIR}{SUB_DATA_DIR}/images/{image_id}.jpg"

    def date_converter(string_date):
        year, month, day = tuple(string_date.split("T")[0].split('-'))
        return date(year=int(year), month=int(month), day=int(day))

    ret: [] = []
    last_row = {}
    i = 0
    for row in raw_data:
        if row["image"]["value"].endswith("jpg") or row["image"]["value"].endswith("jpeg"):
            try:
                new_row = {
                    "titre": row["peintureLabel"]["value"],
                    "date": date_converter(row["inceptionLabel"]["value"]),
                    "createur": row["createurLabel"]["value"],
                    "location": row["locationLabel"]["value"],
                    "origin_path": row["image"]["value"]
                }

                is_new = True

                for key in new_row.keys():
                    if new_row[key] == last_row.get(key, ""):
                        is_new = False

                if is_new:
                    ret.append(
                        new_row
                    )
                    new_row["local_path"] = format_import_data_path(i)
                    i += 1

                    last_row = new_row

            except ValueError as e:
                pass

    return ret

def get_wikidata_paintings(n = 2500):
    def get_results(endpoint_url, query):
        sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        return sparql.query().convert()
    return get_results(endpoint_url, """
    SELECT DISTINCT ?peintureLabel ?createurLabel  ?inceptionLabel ?locationLabel ?image WHERE {
      ?peinture wdt:P31 wd:Q11060274;
        wdt:P170 ?createur;
        wdt:P276 ?location;
        wdt:P571 ?inception;
        wdt:P18 ?image.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
    }
    ORDER BY DESC(?peintureLabel)
    LIMIT""" + str(n))["results"]["bindings"]

## Importation des données

In [70]:
from pandas import json_normalize

raw_data = get_wikidata_paintings()

data_dict = format_data(raw_data)

dataframe = json_normalize(data_dict)

dataframe.reset_index()

dataframe.apply(download_images, axis=1)

dataframe.to_json(f"{PROJECT_DATA_DIR}{SUB_DATA_DIR}/data.json", orient="index")

dataframe

,titre,date,createur,location,origin_path,local_path
0,Épilation,1701-01-01,http://www.wikidata.org/.well-known/genid/589b...,musée Carnavalet,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/0.jpg
1,"« Le Seuil », de Venise, série de douze eaux-f...",1879-01-01,James Abbott McNeill Whistler,Metropolitan Museum of Art,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/1.jpg
2,vue de l'entrée de la rivière de Sierra-Leone,1776-01-01,Nicolas-Marie Ozanne,Bibliothèque nationale de France,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/2.jpg
3,le chateau de Carnarvon,1775-01-01,William Byrne,Bibliothèque nationale du pays de Galles,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/3.jpg
4,chevet de l'église Saint-Martin-sur-Renelle,1860-01-01,Charles Meryon,National Gallery of Art,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/4.jpg
...,...,...,...,...,...,...
302,Q98209359,1569-01-01,Maarten van Heemskerck,Cabinet des Estampes de la Bibliothèque royale...,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/302.jpg
303,Q98209153,1564-01-01,Philippe Galle,KBR,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/303.jpg
304,Q98209143,1569-01-01,Maarten van Heemskerck,Cabinet des Estampes de la Bibliothèque royale...,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/304.jpg
305,Q98209131,1566-01-01,Philippe Galle,KBR,http://commons.wikimedia.org/wiki/Special:File...,../../data/projet/import/images/305.jpg


## Lecture du json en dataframe

In [0]:
from datetime import date

dataframe = pd.read_json(f"{PROJECT_DATA_DIR}{SUB_DATA_DIR}/data.json", orient="index")

dataframe.date = pd.to_datetime(dataframe.date, unit="ms").apply(lambda x: x.date()) # obligé de faire l'import en 2 étapes pour la date, car panda n'a pas de fonction `to_date()`

dataframe

## DETERMINATION DE LA COULEUR PRÉDOMINANTE SUR 1 IMAGE

In [ ]:
#######  #######
import numpy as np
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Charger l'image
img = cv2.imread('../../data/projet/import/images/8.jpg')

# Convertir l'image en matrice de pixels
pixels = np.float32(img.reshape(-1, 3))
print(pixels)
# Appliquer K-means avec 2 clusters
kmeans = KMeans(n_clusters=2, n_init=10).fit(pixels)
print("kmeans :", kmeans)
# Obtenir les centroids des clusters
centroids = kmeans.cluster_centers_
print("Centroids :", centroids)
# Déterminer la couleur prédominante
predominant_color = np.uint8(centroids[0])
print("Couleur prédominante :", predominant_color)
# Afficher la couleur prédominante sur l'image
plt.imshow([[(predominant_color / 255)]])
plt.show()

## DETERMINATION DE LA COULEUR PRÉDOMINANTE SUR TOUTES LES IMAGES

In [0]:
import numpy as np
import cv2
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import os
import swifter


def func(row):
    img = cv2.imread(row.local_path)

    # Convertir l'image en matrice de pixels
    pixels = np.float32(img.reshape(-1, 3))

    # Appliquer K-means avec 2 clusters
    kmeans = KMeans(n_clusters=2, n_init=10).fit(pixels)

    # Obtenir les centroids des clusters
    centroids = kmeans.cluster_centers_

    # Déterminer la couleur prédominante
    predominant_color = np.uint8(centroids[0])
    print("Couleur prédominante :", img, predominant_color)
    return predominant_color


clusters = dataframe.swifter.apply(func, axis=1)


# Afficher la couleur prédominante sur l'image
# plt.imshow([[(predominant_color / 255)]])
# plt.show()

## CLUSTERING DES IMAGES EN FONCTION DE LA COULEUR PRÉDOMINANTE

In [ ]:
import numpy as np
import cv2
from sklearn.cluster import KMeans

# Charger les images
images = ['../../data/projet/import/images/1.jpg', '../../data/projet/import/images/2.jpg',
          '../../data/projet/import/images/3.jpg']

# Créer une liste pour stocker les couleurs prédominantes de chaque image
predominant_colors = []

for image in images:
    # Charger l'image
    img = cv2.imread(image)

    # Convertir l'image en matrice de pixels
    pixels = np.float32(img.reshape(-1, 3))

    # Appliquer K-means avec 2 clusters
    kmeans = KMeans(n_clusters=2, n_init=10).fit(pixels)

    # Obtenir les centroids des clusters
    centroids = kmeans.cluster_centers_

    # Déterminer la couleur prédominante
    predominant_color = np.uint8(centroids[0])

    # Ajouter la couleur prédominante à la liste
    predominant_colors.append(predominant_color)

# Convertir la liste en tableau numpy
predominant_colors = np.array(predominant_colors)

# Appliquer K-means avec 2 clusters sur les couleurs prédominantes
kmeans = KMeans(n_clusters=2).fit(predominant_colors)

# Obtenir les labels de cluster pour chaque couleur prédominante
cluster_labels = kmeans.labels_

# Imprimer les labels de cluster pour chaque image
for i, image in enumerate(images):
    print("L'image {} appartient au cluster {}".format(image, cluster_labels[i]))